In [ ]:
import panel as pn
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.sampledata import stocks

pn.extension()

This example is meant to make it easy to compare and contrast the different APIs Panel provides to declare apps and dashboards. Specifically it demonstrates how to use the ``interact`` function, the reactive programming model and explicit callbacks to achieve the same dynamic stock explorer app.

### Interact

In the ``interact`` model the widgets are automatically generated from the arguments to the function or by providing additional hints to the ``interact`` call. This is an easy way to generate a simple app but it is not very explicit and to compose the different components in a custom way it is necessary to unpack the layout returned by the ``interact`` call:

In [ ]:
tickers = ['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT']

def get_stock(ticker):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    df.plot.line('date', 'close', figsize=(10, 6))
    fig = plt.gcf()
    plt.close()
    return fig

interact = pn.interact(get_stock, ticker=tickers)
pn.Row(
    pn.Column(
        '## Stock Explorer Matplotlib',
        interact[0]
    ),
    interact[1]
)

### Reactive

The reactive programming model is very similar to the ``interact`` function but makes it possible to explicitly declare the inputs to the function using the ``depends`` decorator and makes the layout of the different components more explicit:

In [ ]:
ticker = pn.widgets.Select(options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])

@pn.depends(ticker.param.value)
def get_stock(ticker):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    df.plot.line('date', 'close', figsize=(10, 6))
    fig = plt.gcf()
    plt.close()
    return fig

pn.Row(
    pn.Column(
        '## Stock Explorer Matplotlib',
        ticker
    ),
    get_stock
)

### Callbacks

Declaring an app through explicit callbacks can often afford the most control about precisely how the different components of the app are updated, however it quickly grows in complexity the more callbacks there are and therefore can easily become unmaintainable. The approach works by defining callbacks using the ``.param.watch`` API which either update or replace the already rendered components.

In [ ]:
ticker = pn.widgets.Select(options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])

def get_stock(ticker):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    df.plot.line('date', 'close', figsize=(10, 6))
    fig = plt.gcf()
    plt.close()
    return fig

row = pn.Row(
    pn.Column(
        '## Stock Explorer Matplotlib',
        ticker
    ),
    get_stock(ticker.options[0])
)

def update(event):
    row[1].object = get_stock(event.new)
    # To replace component use:
    # row[1] = get_stock(event.new)
    
ticker.param.watch(update, 'value')

row.servable()